This algorithm aims to compute the evolutionary strategy of delayed dispersal of our model. To do so, we evaluate the inclusive fitness change when we increase slightly the level of delayed dispersal. If this change is positive, we increment level of delayed dispesal by a small amount. We do this process for the level of delayed dispersal in low-quality group h_x and high quality groups h_y. When compute the inclusive change of both delayed dispersal strategies until they are in a tolerance of 0, or are on the boundary of the [0,1]^2 square. We assume we found the evolutionary equilibrium (h_x,h_y) when they are in a tolerance of 0.

# Getting started: libraries, parameters and functions

In [1]:
#Libraries
using LinearAlgebra
using Plots
using LaTeXStrings
using Profile
using Random
using CSV
using DataFrames


(process:18876): GLib-GIO-WARNING **: 15:12:04.657: Unexpectedly, UWP app `Microsoft.OutlookForWindows_1.2024.625.600_x64__8wekyb3d8bbwe' (AUMId `Microsoft.OutlookForWindows_8wekyb3d8bbwe!Microsoft.OutlookforWindows') supports 4 extensions but has no verbs


In [2]:
#Parameters

#survival
s_u = 0.5
s_b0 = 0.5
s_b1 = 0.5
s_a0 = 0.5
s_a1 = 0.5

#fecundity
n_0 = 3
n_1 = 3

phi = 0.1
M = 1

#Male and female reproduction:
theta_0(b_0,b_1) = (1-phi)*b_0/(b_0+b_1)
theta_1(b_0,b_1) = (1-phi)*b_1/(b_0+b_1)
psi_0(b_0,b_1) = 1+phi+theta_0(b_0,b_1)
psi_1(b_0,b_1) = 1+phi+theta_1(b_0,b_1)

#competition
p_u = 0.5 #floater ability
p_a = 0.25 #helper ability
a = 2 #strength of the density
#Actual probability to find a breeding spot during a time step
p_u_t(b_0,b_1) = p_u/(1+a*(b_0+b_1))
p_a_t(b_0,b_1) = p_a/(1+a*(b_0+b_1))

#Cooperation levels
h_0 = 0
h_1 = 0
h_0_bar = 0
h_1_bar = 0

#Parameter dictionary
parameters = Dict("n_0"=>n_0,"n_1"=>n_1,"s_u"=>s_u,"s_a0"=>s_a0,"s_a1"=>s_a1,"s_b0"=>s_b0,"s_b1"=>s_b1,"phi"=>phi,"M"=>M,"a"=>a,"p_u"=>p_u,"p_a"=>p_a,"h_0" =>h_0, "h_1" =>h_1,"h_0_bar" =>h_0_bar, "h_1_bar" =>h_1_bar)


Dict{String, Real} with 16 entries:
  "s_b1"    => 0.5
  "s_u"     => 0.5
  "n_1"     => 3
  "s_b0"    => 0.5
  "p_u"     => 0.5
  "a"       => 2
  "h_1_bar" => 0
  "M"       => 1
  "phi"     => 0.1
  "h_0"     => 0
  "s_a0"    => 0.5
  "h_0_bar" => 0
  "p_a"     => 0.25
  "s_a1"    => 0.5
  "n_0"     => 3
  "h_1"     => 0

Usefull functions: Wild type population matrix A, mutant population type matrix M, computations of the eigenvectors mu (right) and nu (left) and relatedness coefficient.

In [3]:
#Population matrix
function B(b_0,b_1,parameters) #x and y are the number of breeders in low quality (x) and high quality (y) groups
    #First we retrieve the parameters from the dictionnary parameters
    n_0 = parameters["n_0"]
    n_1 = parameters["n_1"]
    s_u = parameters["s_u"]
    s_a0 = parameters["s_a0"]
    s_a1 = parameters["s_a1"]
    s_b0 = parameters["s_b0"]
    s_b1 = parameters["s_b1"]
    p_u = parameters["p_u"]
    p_a = parameters["p_a"]
    a = parameters["a"]
    M = parameters["M"]
    phi = parameters["phi"]
    h_0 = parameters["h_0"]
    h_1 = parameters["h_1"]
    h_0_bar = parameters["h_0_bar"]
    h_1_bar = parameters["h_1_bar"]

    #We use the parameters to compute the functions
    theta_0(b_0,b_1) = (1-phi)*b_0/(b_0+b_1)
    theta_1(b_0,b_1) = (1-phi)*b_1/(b_0+b_1)
    psi_0(b_0,b_1) = 1+phi+theta_0(b_0,b_1)
    psi_1(b_0,b_1) = 1+phi+theta_1(b_0,b_1)
    T_0(h_0) = 1-exp(-h_0*n_0)
    T_1(h_1) = exp(-h_1*n_1) 
    p_u_t(b_0,b_1) = p_u/(1+a*(b_0+b_1))
    p_a_t(b_0,b_1) = p_a/(1+a*(b_0+b_1))
    
    #Finally, we return the matrix A(x,y):
    return [s_u*(1-p_u_t(b_0,b_1)) 0 0 (1-h_0)*s_u*(1-p_u_t(b_0,b_1))+h_0*(T_0(h_0_bar)*s_a1+(1-T_0(h_0_bar))*s_a0)*(1-p_a_t(b_0,b_1)) (1-h_1)*s_u*(1-p_u_t(b_0,b_1))+h_1*(T_1(h_1_bar)*s_a0+(1-T_1(h_1_bar))*s_a1)*(1-p_a_t(b_0,b_1));
    s_u*p_u_t(b_0,b_1) (1-T_0(h_0_bar))*s_b0 T_1(h_1_bar)*s_b0 (1-h_0)*s_u*p_u_t(b_0,b_1)+h_0*(T_0(h_0_bar)*s_a1+(1-T_0(h_0_bar))*s_a0)*p_a_t(b_0,b_1) (1-h_1)*s_u*p_u_t(b_0,b_1)+h_1*(T_1(h_1_bar)*s_a0+(1-T_1(h_1_bar))*s_a1)*p_a_t(b_0,b_1);
    0 T_0(h_0_bar)*s_b1 (1-T_1(h_1_bar))*s_b1 0 0 ;       
    s_u*p_u_t(b_0,b_1)*n_0 (1-T_0(h_0_bar))*s_b0*n_0 T_1(h_1_bar)*s_b0*n_0 ((1-h_0)*s_u*p_u_t(b_0,b_1)+h_0*(T_0(h_0_bar)*s_a1+(1-T_0(h_0_bar))*s_a0)*p_a_t(b_0,b_1))*n_0 ((1-h_1)*s_u*p_u_t(b_0,b_1)+h_1*(T_1(h_1_bar)*s_a0+(1-T_1(h_1_bar))*s_a1)*p_a_t(b_0,b_1))*n_0;  
    0 T_0(h_0_bar)*s_b1*n_1 (1-T_1(h_1_bar))*s_b1*n_1 0 0
    ]
end


function Mutant(b_0,b_1,parameters)
    #As for B, we retrieve the parameters from the dictionnary and compute the functions
   
  
    
    n_0 = parameters["n_0"]
    n_1 = parameters["n_1"]
    s_u = parameters["s_u"]
    s_a0 = parameters["s_a0"]
    s_a1 = parameters["s_a1"]
    s_b0 = parameters["s_b0"]
    s_b1 = parameters["s_b1"]
    p_u = parameters["p_u"]
    p_a = parameters["p_a"]
    a = parameters["a"]
    M = parameters["M"]
    phi = parameters["phi"]
    h_0 = parameters["h_0"]
    h_1 = parameters["h_1"]
    h_0_bar = parameters["h_0_bar"]
    h_1_bar = parameters["h_1_bar"]
    theta_0(b_0,b_1) = (1-phi)*b_0/(b_0+b_1)
    theta_1(b_0,b_1) = (1-phi)*b_1/(b_0+b_1)
    psi_0(b_0,b_1) = 1+phi+theta_0(b_0,b_1)
    psi_1(b_0,b_1) = 1+phi+theta_1(b_0,b_1)
    T_0(h_0) = 1-exp(-h_0*n_0)
    T_1(h_1) = exp(-h_1*n_1)
    p_u_t(b_0,b_1) = p_u/(1+a*(b_0+b_1))
    p_a_t(b_0,b_1) = p_a/(1+a*(b_0+b_1))
    
    #We return the value of the mutant matrix M(b_0,b_1)
    
    return [s_u*(1-p_u_t(b_0,b_1)) 0 0 (1-h_0)*s_u*(1-p_u_t(b_0,b_1))+h_0*(T_0(h_0_bar)*s_a1+(1-T_0(h_0_bar))*s_a0)*(1-p_a_t(b_0,b_1)) (1-h_1)*s_u*(1-p_u_t(b_0,b_1))+h_1*(T_1(h_1_bar)*s_a0+(1-T_1(h_1_bar))*s_a1)*(1-p_a_t(b_0,b_1));
    s_u*p_u_t(b_0,b_1) (1-T_0(h_0_bar))*s_b0 T_1(h_1_bar)*s_b0 (1-h_0)*s_u*p_u_t(b_0,b_1)+h_0*(T_0(h_0_bar)*s_a1+(1-T_0(h_0_bar))*s_a0)*p_a_t(b_0,b_1) (1-h_1)*s_u*p_u_t(b_0,b_1)+h_1*(T_1(h_1_bar)*s_a0+(1-T_1(h_1_bar))*s_a1)*p_a_t(b_0,b_1); 
    0 T_0(h_0_bar)*s_b1 (1-T_1(h_1_bar))*s_b1 0 0;
        
    s_u*p_u_t(b_0,b_1)*n_0/2*psi_0(b_0,b_1) (1-T_0(h_0_bar))*s_b0*n_0/2*psi_0(b_0,b_1)+T_0(h_0_bar)*s_b1*n_0/2*theta_0(b_0,b_1) T_1(h_1_bar)*s_b0*n_0/2*psi_0(b_0,b_1)+(1-T_1(h_1_bar))*s_b1*n_0/2*theta_0(b_0,b_1) ((1-h_0)*s_u*p_u_t(b_0,b_1)+h_0*(T_0(h_0_bar)*s_a1+(1-T_0(h_0_bar))*s_a0)*p_a_t(b_0,b_1))*n_0/2*psi_0(b_0,b_1) ((1-h_1)*s_u*p_u_t(b_0,b_1)+h_1*(T_1(h_1_bar)*s_a0+(1-T_1(h_1_bar))*s_a1)*p_a_t(b_0,b_1))*n_0/2*psi_0(b_0,b_1);
        
    s_u*p_u_t(b_0,b_1)*n_1/2*theta_1(b_0,b_1) T_0(h_0_bar)*s_b1*n_1/2*psi_1(b_0,b_1)+(1-T_0(h_0_bar))*s_b0*n_1/2*theta_1(b_0,b_1) (1-T_1(h_1_bar))*s_b1*n_1/2*psi_1(b_0,b_1)+T_1(h_1_bar)*s_b0*n_1/2*theta_1(b_0,b_1) ((1-h_0)*s_u*p_u_t(b_0,b_1)+h_0*(T_0(h_0_bar)*s_a1+(1-T_0(h_0_bar))*s_a0)*p_a_t(b_0,b_1))*n_1/2*theta_1(b_0,b_1) ((1-h_1)*s_u*p_u_t(b_0,b_1)+h_1*(T_1(h_1_bar)*s_a0+(1-T_1(h_1_bar))*s_a1)*p_a_t(b_0,b_1))*n_1/2*theta_1(b_0,b_1)
    ]
end



function right_eigenvector(parameters)
    #We suppose a random value for the equilibrium of the mutant matrix
    mu = rand!(zeros(5,1))
    #And compute the equilibrium value until we find a value that satisfies the tolerance (1e^-8)
    while sum(abs.(mu-B(mu[2],mu[3],parameters)*mu))>1e-8
        mu = B(mu[2],mu[3],parameters)*mu
    end
    return mu
end

function left_eigenvector(parameters,mu)
    #We suppose a random value for the left eigenvalue of the mutant matrix
    nu = rand!(zeros(1,5))
    #And compute the left eigenvector value until we find a value that satisfies the tolerance (1e^-8)
    while sum(abs.(nu-nu*Mutant(mu[2],mu[3],parameters)))>1e-8
        nu = nu*Mutant(mu[2],mu[3],parameters)
        nu /= sum(nu)
    end
    return nu
end

function R_parent(parameters)
    return 1/2*(1+parameters["phi"]) 
end

function R_siblings_0(parameters)
    n_0 = parameters["n_0"] 
    M = parameters["M"]
    phi = parameters["phi"]
    return  (1-1/n_0)*(phi^2+phi*(1-phi)+(1-phi)^2*(1/4+1/(4*M)))+1/n_0
end

function R_siblings_1(parameters)
    M = parameters["M"]
    phi = parameters["phi"]
    n_1 = parameters["n_1"]   
    return  (1-1/n_1)*(phi^2+phi*(1-phi)+(1-phi)^2*(1/4+1/(4*M)))+1/n_1
end;

Now that we have our useful function, we can compute the evolution of the delayed dispersal rates h_x, h_y

In [4]:
function evol_dd(parameters,ctrl)
    #initialization
    dW_0 = 1
    dW_1 = 1
    sign0 = 1
    sign1 = 1
    #delta h_0, delta h_1
    dh0 = 1e-4
    dh1 = 1e-4
    tol = 1e-8
    parameters["h_0"] = 0
    parameters["h_0_bar"] = 0
    parameters["h_1"] = 0
    parameters["h_1_bar"] = 0
    i = 0        
    reason = "too many iterations"

    h0_mem = 10
    h1_mem = 10
    dw0_mem = 10
    dw1_mem = 10
    #Evolution
    #We run the simulation for a sufficient amount of time until the simulation stop
    while i<1000000
        reason = "too many iterations"

        #Eigenvectors mu and nu
        mu = right_eigenvector(parameters)
        nu = left_eigenvector(parameters,mu)
        #h_0 evolution
        parameters["h_0"] += dh0 #We assume h_0 is slightly higher and evaluate the change in the fitness of the individual, its breeder, the other late dispersers
        W_indiv_0 = nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]
        parameters["h_0"] -= 2*dh0
        W_indiv_0 -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]
        parameters["h_0"] += dh0

        parameters["h_0_bar"] += dh0
        W_parent_0 = nu*Mutant(mu[2],mu[3],parameters)*[0,mu[2],0,0,0]
        W_sibs_0 = nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]

        parameters["h_0_bar"] -= 2*dh0
        W_parent_0-= nu*Mutant(mu[2],mu[3],parameters)*[0,mu[2],0,0,0]
        W_sibs_0-= nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]
        parameters["h_0_bar"] += dh0
        
        #h_1 same process as h_0
        parameters["h_1"] += dh1
        W_indiv_1 = nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,0,mu[5]]
        parameters["h_1"] -= 2*dh1
        W_indiv_1 -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,0,mu[5]]
        parameters["h_1"] += dh1

        parameters["h_1_bar"] += dh1
        W_parent_1 = nu*Mutant(mu[2],mu[3],parameters)*[0,0,mu[3],0,0]
        W_sibs_1 = nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,0,mu[5]]

        parameters["h_1_bar"] -= 2*dh1
        W_parent_1 -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,mu[3],0,0]
        W_sibs_1 -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,0,mu[5]]
        parameters["h_1_bar"] += dh1
        
        
        if ctrl=="sub"
            dW_0 = R_parent(parameters)*W_parent_0[1]+W_indiv_0[1]+R_siblings_0(parameters)*W_sibs_0[1]
            dW_1 = R_parent(parameters)*W_parent_1[1]+W_indiv_1[1]+R_siblings_1(parameters)*W_sibs_1[1]
        else
            dW_0 = W_parent_0[1]+(W_sibs_0[1]+W_indiv_0[1])*R_parent(parameters)
            dW_1 = W_parent_1[1]+(W_sibs_1[1]+W_indiv_1[1])*R_parent(parameters)
        end
        h0_mem = parameters["h_0"]
        h1_mem = parameters["h_1"]
        dw0_mem = dW_0
        dw1_mem = dW_1
        parameters["h_0"] += dW_0*50
        parameters["h_0_bar"] += dW_0*50
        parameters["h_1"] += dW_1*50
        parameters["h_1_bar"] += dW_1*50

        i+=1
        #We keep both values between 0 and 1 since they are probabilities
        parameters["h_0"] = clamp(parameters["h_0"],0,1)
        parameters["h_0_bar"] = clamp(parameters["h_0_bar"],0,1)
        parameters["h_1"] = clamp(parameters["h_1"],0,1)
        parameters["h_1_bar"] = clamp(parameters["h_1_bar"],0,1)
        
        #the algorithm stops if:
        #1. The inclusive fitness change is within a tolerance of 0
        if abs(dW_0)<tol && abs(dW_1)<tol
            #If we run for too long, we break the loop
            reason = "css found"
            break
        end
        #2. Both values are repeated (this can happen at the border of the [0,1]x[0,1] square)
        
        if parameters["h_0"] == h0_mem && parameters["h_1"] == h1_mem
             reason = "stagnation"
            break
        end
       
        #3. One value is repeated while the other remains an internal equilibrium: the change for both values is thus
        #null/not significant
        if parameters["h_0"] == h0_mem && abs(dW_1)<tol && abs(dw1_mem)<tol
            reason = "limit"
            break
        end
        
        if parameters["h_1"] == h1_mem && abs(dW_0)<tol && abs(dw0_mem)<tol
            reason = "limit"
            break
        end    
        
    end
    #Evaluation of group augmentation and cost of delayed dispersal at evolutionary equilibrium:
    eq = right_eigenvector(parameters)
    mu = eq
    nu = left_eigenvector(parameters,mu)
    #Group size effect
    GS = (1-p_a_t(mu[2],mu[3]))*nu[1]+p_a_t(mu[2],mu[3])*(nu[2]+(1+phi)*n_0/2*nu[4]+(1-phi)*mu[2]/(mu[2]+mu[3])*n_0/2*nu[4]+(1-phi)*mu[3]/(mu[2]+mu[3])*n_1/2*nu[5])
    #Probability of establishment
    P2 = parameters["s_u"]*p_u_t(mu[2],mu[3])/(1-parameters["s_u"]*(1-p_u_t(mu[2],mu[3])))
    #Cost of delaying dispersal
    parameters["h_0"] += dh0
    W_indiv_0 = nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]
    parameters["h_0"] -= 2*dh0
    W_indiv_0 -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]
    parameters["h_0"] += dh0
    cost = W_indiv_0[1]
    return parameters, P2, GS, cost, reason
end

evol_dd (generic function with 1 method)

# Simulations

## Figure 2: impact of selfing rate phi on delayed dispersal

In [ ]:
#We first set the parameters
parameters["s_a0"] = 0.5
parameters["s_a1"] = 0.6
parameters["s_u"] = 0.5
parameters["s_b0"] = 0.5
parameters["s_b1"] = 0.5
parameters["n_0"] = 3
parameters["n_1"] = 3
parameters["phi"] = 0.1
N = 50

#We initialize our data
phi_csv = zeros(N^2)
dn_csv = zeros(N^2)
h0_csv = zeros(N^2)
h1_csv = zeros(N^2)
P_csv = zeros(N^2)
GS_csv = zeros(N^2)
cost_csv = zeros(N^2)
parameters_csv = Array{Dict}(undef, N^2)
reason_csv = Array{String}(undef, N^2)
j = 1

#We loop over the parameters we are interested in, here fecundity benefits and phi
for dn in range(0,2,N)
    parameters["n_1"] = parameters["n_0"]+dn
    for phi in range(0,1,N)
        parameters["phi"] = phi
        parameters, P2, GS, cost, reason = evol_dd(parameters,"sub")
        h0_csv[j] = parameters["h_0"]
        h1_csv[j] = parameters["h_1"]
        phi_csv[j] = phi
        dn_csv[j] = dn
        GS_csv[j] = GS
        cost_csv[j] = cost
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
#Finally, we write the result in a csv file
csv_data = DataFrame(dn = dn_csv, phi = phi_csv, h_0 = h0_csv, h_1 = h1_csv, cost=cost_csv, GS = GS_csv, parameters = parameters_csv, reason = reason_csv )

CSV.write("result_phi_n_sub_00.csv",csv_data)

In [6]:
CSV.write("result_phi_n_sub_00.csv",csv_data)

"result_phi_n_sub_00.csv"

## Figure 3: relationship between delayed dispersal and probability of establishment

Panel A: fecundity

In [6]:
parameters["s_b0"] = 0.5
parameters["s_b1"] = 0.5
parameters["s_a0"] = 0.5
parameters["s_a1"] = 0.5
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 5
n_csv = zeros(N^2)
dn_csv = zeros(N^2)
h0_csv = zeros(N^2)
h1_csv = zeros(N^2)
P2_csv = zeros(N^2)
parameters_csv = Array{Dict}(undef, N^2)
reason_csv = Array{String}(undef, N^2)
j = 1
for n in range(3,7,N)
    parameters["n_0"] = n
    for dn in range(0,5,N)
        parameters["n_1"] = parameters["n_0"]+dn
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        h0_csv[j] = parameters["h_0"]
        h1_csv[j] = parameters["h_1"]
        n_csv[j] = n
        dn_csv[j] = dn
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(dn = dn_csv, n = n_csv, h_0 = h0_csv, h_1 = h1_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_n_sub.csv",csv_data)

"result_n_sub.csv"

Panel B: survival of the breeders

In [8]:
parameters["n_0"] = 3
parameters["n_1"] = 3
parameters["s_a0"] = 0.5
parameters["s_a1"] = 0.5
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 5
s_csv = zeros(N^2)
ds_csv = zeros(N^2)
h0_csv = zeros(N^2)
h1_csv = zeros(N^2)
P2_csv = zeros(N^2)
parameters_csv = Array{Dict}(undef, N^2)
reason_csv = Array{String}(undef, N^2)
j = 1
for s in range(0.3,0.7,N)
    parameters["s_b0"] = s
    for ds in range(0,0.2,N)
        parameters["s_b1"] = parameters["s_b0"]+ds
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        h0_csv[j] = parameters["h_0"]
        h1_csv[j] = parameters["h_1"]
        s_csv[j] = s
        ds_csv[j] = ds    
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, s = s_csv, h_0 = h0_csv, h_1 = h1_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_s_sub_00.csv",csv_data)

"result_s_sub_00.csv"

Panel C: survival of the subordinates

In [10]:
parameters["n_0"] = 3
parameters["n_1"] = 5
parameters["s_b0"] = 0.5
parameters["s_b1"] = 0.5
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 5
s_csv = zeros(N^2)
ds_csv = zeros(N^2)
h0_csv = zeros(N^2)
h1_csv = zeros(N^2)
P2_csv = zeros(N^2)
parameters_csv = Array{Dict}(undef, N^2)
reason_csv = Array{String}(undef, N^2)
j = 1
for s in range(0.5,0.7,N)
    parameters["s_u"] = s
    parameters["s_a0"] = s
    for ds in range(0,0.15,N)
        parameters["s_a1"] = parameters["s_a0"]+ds
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        h0_csv[j] = parameters["h_0"]
        h1_csv[j] = parameters["h_1"]
        s_csv[j] = s
        ds_csv[j] = ds    
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, s = s_csv, h_0 = h0_csv, h_1 = h1_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_sa1_sub.csv",csv_data)

"result_sa1_sub.csv"

## Figure 4: breeder-subordinate conflict

Fecundity without Group size effect on subordinates survival

In [7]:
parameters["s_b0"] = 0.5
parameters["s_b1"] = 0.5
parameters["s_a0"] = 0.5
parameters["s_a1"] = 0.5
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 10
n_csv = zeros(N*3)
dn_csv = zeros(N*3)
h0_csv = zeros(N*3)
h1_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for n in [4,5,6]
    parameters["n_0"] = n
    for dn in range(0,5,N)
        parameters["n_1"] = parameters["n_0"]+dn
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        h0_csv[j] = parameters["h_0"]
        h1_csv[j] = parameters["h_1"]
        n_csv[j] = n
        dn_csv[j] = dn
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(dn = dn_csv, n = n_csv, h_0 = h0_csv, h_1 = h1_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_n_sub_3_t.csv",csv_data)

"result_n_sub_3_t.csv"

In [8]:
parameters["s_b0"] = 0.5
parameters["s_b1"] = 0.5
parameters["s_a0"] = 0.5
parameters["s_a0"] = 0.5
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 10
n_csv = zeros(N*3)
dn_csv = zeros(N*3)
h0_csv = zeros(N*3)
h1_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for n in [4,5,6]
    parameters["n_0"] = n
    for dn in range(0,5,N)
        parameters["n_1"] = parameters["n_0"]+dn
        parameters, P2, GA, cost, reason = evol_dd(parameters,"bre")
        h0_csv[j] = parameters["h_0"]
        h1_csv[j] = parameters["h_1"]
        n_csv[j] = n
        dn_csv[j] = dn
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(dn = dn_csv, n = n_csv, h_0 = h0_csv, h_1 = h1_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_n_bre_3_t.csv",csv_data)

"result_n_bre_3_t.csv"

Fecundity with GA

In [10]:
parameters["s_b0"] = 0.5
parameters["s_b1"] = 0.5
parameters["s_a0"] = 0.5
parameters["s_a1"] = 0.6
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 10
n_csv = zeros(N*3)
dn_csv = zeros(N*3)
h0_csv = zeros(N*3)
h1_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for n in [4,5,6]
    parameters["n_0"] = n
    for dn in range(0,5,N)
        parameters["n_1"] = parameters["n_0"]+dn
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        h0_csv[j] = parameters["h_0"]
        h1_csv[j] = parameters["h_1"]
        n_csv[j] = n
        dn_csv[j] = dn
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(dn = dn_csv, n = n_csv, h_0 = h0_csv, h_1 = h1_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_b_GA_sub_3_t.csv",csv_data)

"result_b_GA_sub_3_t.csv"

In [11]:
parameters["s_b0"] = 0.5
parameters["s_b1"] = 0.5
parameters["s_a0"] = 0.5
parameters["s_a1"] = 0.6
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 10
n_csv = zeros(N*3)
dn_csv = zeros(N*3)
h0_csv = zeros(N*3)
h1_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for n in [4,5,6]
    parameters["n_0"] = n
    for dn in range(0,5,N)
        parameters["n_1"] = parameters["n_0"]+dn
        parameters, P2, GA, cost, reason = evol_dd(parameters,"bre")
        h0_csv[j] = parameters["h_0"]
        h1_csv[j] = parameters["h_1"]
        n_csv[j] = n
        dn_csv[j] = dn
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(dn = dn_csv, n = n_csv, h_0 = h0_csv, h_1 = h1_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_b_GA_bre_3_t.csv",csv_data)

"result_b_GA_bre_3_t.csv"

Survival without GA

In [ ]:
parameters["n_0"] = 3
parameters["n_1"] = 3
parameters["s_u"] = 0.5
parameters["s_a0"] = 0.5
parameters["s_a1"] = 0.5
parameters["phi"] = 0.1
N = 10
s_csv = zeros(N*3)
ds_csv = zeros(N*3)
h0_csv = zeros(N*3)
h1_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for s in [0.3,0.5,0.7]
    parameters["s_0"] = s
    for ds in range(0,0.2,N)
        parameters["s_b1"] = parameters["s_b0"]+ds
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        h0_csv[j] = parameters["h_0"]
        h1_csv[j] = parameters["h_1"]
        s_csv[j] = s
        ds_csv[j] = ds    
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, s = s_csv, h_0 = h0_csv, h_1 = h1_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_s_sub_3_t.csv",csv_data)

In [ ]:
parameters["n_0"] = 3
parameters["n_1"] = 3
parameters["s_u"] = 0.5
parameters["s_a0"] = 0.5
parameters["s_a1"] = 0.5
parameters["phi"] = 0.1
N = 10
s_csv = zeros(N*3)
ds_csv = zeros(N*3)
h0_csv = zeros(N*3)
h1_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for s in [0.3,0.5,0.7]
    parameters["s_b0"] = s
    for ds in range(0,0.2,N)
        parameters["s_b1"] = parameters["s_b0"]+ds
        parameters, P2, GA, cost, reason = evol_dd(parameters,"bre")
        h0_csv[j] = parameters["h_0"]
        h1_csv[j] = parameters["h_1"]
        s_csv[j] = s
        ds_csv[j] = ds    
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, s = s_csv, h_0 = h0_csv, h_1 = h1_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_s_bre_3_t.csv",csv_data)

Survival with GA

In [9]:
parameters["n_0"] = 3
parameters["n_1"] = 3
parameters["s_u"] = 0.5
parameters["s_a0"] = 0.5
parameters["s_a1"] = 0.6
parameters["phi"] = 0.1
N = 10
s_csv = zeros(N*3)
ds_csv = zeros(N*3)
h0_csv = zeros(N*3)
h1_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for s in [0.3,0.5,0.7]
    parameters["s_b0"] = s
    for ds in range(0,0.2,N)
        parameters["s_b1"] = parameters["s_b0"]+ds
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        h0_csv[j] = parameters["h_0"]
        h1_csv[j] = parameters["h_1"]
        s_csv[j] = s
        ds_csv[j] = ds    
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, s = s_csv, h_0 = h0_csv, h_1 = h1_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_s_GS_sub_3_t.csv",csv_data)

"result_s_GS_sub_3_t.csv"

In [10]:
parameters["n_0"] = 3
parameters["n_1"] = 3
parameters["s_u"] = 0.5
parameters["s_a0"] = 0.5
parameters["s_a1"] = 0.6
parameters["phi"] = 0.1
N = 10
s_csv = zeros(N*3)
ds_csv = zeros(N*3)
h0_csv = zeros(N*3)
h1_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for s in [0.3,0.5,0.7]
    parameters["s_b0"] = s
    for ds in range(0,0.2,N)
        parameters["s_b1"] = parameters["s_b0"]+ds
        parameters, P2, GA, cost, reason = evol_dd(parameters,"bre")
        h0_csv[j] = parameters["h_0"]
        h1_csv[j] = parameters["h_1"]
        s_csv[j] = s
        ds_csv[j] = ds    
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, s = s_csv, h_0 = h0_csv, h_1 = h1_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_s_GS_bre_3_t.csv",csv_data)

"result_s_GS_bre_3_t.csv"

Survival for the subordinates

In [7]:
parameters["n_0"] = 3
parameters["n_1"] = 5
parameters["s_b0"] = 0.5
parameters["s_b1"] = 0.5
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 10
s_csv = zeros(N*3)
ds_csv = zeros(N*3)
h0_csv = zeros(N*3)
h1_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for s in [0.5,0.6,0.7]
    parameters["s_u"] = s
    parameters["s_a0"] = s
    for ds in range(0,0.15,N)
        parameters["s_a1"] = parameters["s_a0"]+ds
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        h0_csv[j] = parameters["h_0"]
        h1_csv[j] = parameters["h_1"]
        s_csv[j] = s
        ds_csv[j] = ds    
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, s = s_csv, h_0 = h0_csv, h_1 = h1_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_sa1_sub_3_t.csv",csv_data)

"result_sa1_sub_3_t.csv"

In [8]:
parameters["n_0"] = 3
parameters["n_1"] = 5
parameters["s_b0"] = 0.5
parameters["s_b1"] = 0.5
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 10
s_csv = zeros(N*3)
ds_csv = zeros(N*3)
h0_csv = zeros(N*3)
h1_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for s in [0.5,0.6,0.7]
    parameters["s_u"] = s
    parameters["s_a0"] = s
    for ds in range(0,0.15,N)
        parameters["s_a1"] = parameters["s_a0"]+ds
        parameters, P2, GA, cost, reason = evol_dd(parameters,"bre")
        h0_csv[j] = parameters["h_0"]
        h1_csv[j] = parameters["h_1"]
        s_csv[j] = s
        ds_csv[j] = ds    
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, s = s_csv, h_0 = h0_csv, h_1 = h1_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_sa1_bre_3_t.csv",csv_data)

"result_sa1_bre_3_t.csv"

## Supplementary data:

In [11]:
parameters["n_0"] = 3
parameters["n_1"] = 3
parameters["s_b0"] = 0.5
parameters["s_b1"] = 0.5
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 10
s_csv = zeros(N*3)
ds_csv = zeros(N*3)
h0_csv = zeros(N*3)
h1_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for s in [0.5,0.6,0.7]
    parameters["s_u"] = s
    for ds in range(0,0.2,N)
        parameters["s_a0"] = parameters["s_u"]+ds
        parameters["s_a1"] = parameters["s_u"]+ds
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        h0_csv[j] = parameters["h_0"]
        h1_csv[j] = parameters["h_1"]
        s_csv[j] = s
        ds_csv[j] = ds    
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, s = s_csv, h_0 = h0_csv, h_1 = h1_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_sa_sub_3_t.csv",csv_data)

"result_p_sa_sub.csv"

In [ ]:
parameters["n_0"] = 3
parameters["n_1"] = 3
parameters["s_b0"] = 0.5
parameters["s_b1"] = 0.5
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 10
s_csv = zeros(N*3)
ds_csv = zeros(N*3)
h0_csv = zeros(N*3)
h1_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for s in [0.5,0.6,0.7]
    parameters["s_u"] = s
    for ds in range(0,0.2,N)
        parameters["s_a0"] = parameters["s_u"]+ds
        parameters["s_a1"] = parameters["s_u"]+ds
        parameters, P2, GA, cost, reason = evol_dd(parameters,"bre")
        h0_csv[j] = parameters["h_0"]
        h1_csv[j] = parameters["h_1"]
        s_csv[j] = s
        ds_csv[j] = ds    
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, s = s_csv, h_0 = h0_csv, h_1 = h1_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_sa_bre_3_t.csv",csv_data)

Breeder-subordinates conflict: phi and fecundity

In [12]:
parameters["s_a0"] = 0.5
parameters["s_a1"] = 0.6
parameters["s_u"] = 0.5
parameters["s_b0"] = 0.5
parameters["s_b1"] = 0.5
parameters["n_0"] = 3
parameters["n_1"] = 3
parameters["phi"] = 0.1
N = 10
phi_csv = zeros(N*3)
dn_csv = zeros(N*3)
h0_csv = zeros(N*3)
h1_csv = zeros(N*3)
P_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
GS_csv = zeros(N*3)
cost_csv = zeros(N*3)
j = 1
for dn in [2,3,4]
    parameters["n_1"] = parameters["n_0"]+dn
    for phi in range(0,1,N)
    parameters["phi"] = phi
        parameters, P2, GS, cost, reason = evol_dd(parameters,"sub")
        h0_csv[j] = parameters["h_0"]
        h1_csv[j] = parameters["h_1"]
        phi_csv[j] = phi
        db_csv[j] = db
        GS_csv[j] = GS
        cost_csv[j] = cost
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(db = db_csv, phi = phi_csv, h_0 = h0_csv, h_1 = h1_csv, cost=cost_csv, GS = GS_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_phi_b_GS_sub_3_t.csv",csv_data)

LoadError: SystemError: opening file "result_phi_b_GA_bre3.csv": Permission denied

In [ ]:
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.6
parameters["s_u"] = 0.5
parameters["s_x"] = 0.5
parameters["s_y"] = 0.5
parameters["b_x"] = 3
parameters["b_y"] = 3
parameters["phi"] = 0.1
N = 30
phi_csv = zeros(N*3)
db_csv = zeros(N*3)
hx_csv = zeros(N*3)
hy_csv = zeros(N*3)
P_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
GA_csv = zeros(N*3)
cost_csv = zeros(N*3)
j = 1
for db in [2,3,4]
    parameters["b_y"] = parameters["b_x"]+db
    for phi in range(0,1,N)
    parameters["phi"] = phi
        parameters, P2, GA, cost, reason = evol_dd(parameters,"bre")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        phi_csv[j] = phi
        db_csv[j] = db
        GA_csv[j] = GA
        cost_csv[j] = cost
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(db = db_csv, phi = phi_csv, h_x = hx_csv, h_y = hy_csv, cost=cost_csv, GA = GA_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_phi_b_GA_bre_3.csv",csv_data)

Breeder-subordinate conflict: survival and phi

In [10]:
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.6
parameters["s_u"] = 0.5
parameters["s_x"] = 0.5
parameters["s_y"] = 0.5
parameters["b_x"] = 3
parameters["b_y"] = 3
parameters["phi"] = 0.1
N = 10
phi_csv = zeros(N*3)
ds_csv = zeros(N*3)
hx_csv = zeros(N*3)
hy_csv = zeros(N*3)
P_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
GA_csv = zeros(N*3)
cost_csv = zeros(N*3)
j = 1
for ds in [0.2,0.3,0.4]
    parameters["s_y"] = parameters["s_x"]+ds
    for phi in range(0,1,N)
    parameters["phi"] = phi
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        phi_csv[j] = phi
        ds_csv[j] = ds
        GA_csv[j] = GA
        cost_csv[j] = cost
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, phi = phi_csv, h_x = hx_csv, h_y = hy_csv, cost=cost_csv, GA = GA_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_phi_s_GA_sub_3.csv",csv_data)

"result_phi_s_GA_sub_4.csv"

In [10]:
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.6
parameters["s_u"] = 0.5
parameters["s_x"] = 0.5
parameters["s_y"] = 0.5
parameters["b_x"] = 3
parameters["b_y"] = 3
parameters["phi"] = 0.1
N = 20
phi_csv = zeros(N*3)
ds_csv = zeros(N*3)
hx_csv = zeros(N*3)
hy_csv = zeros(N*3)
P_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
GA_csv = zeros(N*3)
cost_csv = zeros(N*3)
j = 1
for ds in [0.2,0.3,0.4]
    parameters["s_y"] = parameters["s_x"]+ds
    for phi in range(0,1,N)
    parameters["phi"] = phi
        parameters, P2, GA, cost, reason = evol_dd(parameters,"bre")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        phi_csv[j] = phi
        ds_csv[j] = ds
        GA_csv[j] = GA
        cost_csv[j] = cost
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, phi = phi_csv, h_x = hx_csv, h_y = hy_csv, cost=cost_csv, GA = GA_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_phi_s_GA_bre3.csv",csv_data)

"result_phi_s_GA_bre3.csv"

In [110]:
data = CSV.read("result_n_sub.csv",DataFrame)
data[]

┌ Warning: thread = 1 warning: only found 1 / 8 columns around data row: 1. Filling remaining columns with `missing`
└ @ CSV C:\Users\Alan\.julia\packages\CSV\0Elut\src\file.jl:577
┌ Warning: thread = 1 warning: only found 1 / 8 columns around data row: 2. Filling remaining columns with `missing`
└ @ CSV C:\Users\Alan\.julia\packages\CSV\0Elut\src\file.jl:577
┌ Warning: thread = 1 warning: only found 1 / 8 columns around data row: 3. Filling remaining columns with `missing`
└ @ CSV C:\Users\Alan\.julia\packages\CSV\0Elut\src\file.jl:577
┌ Warning: thread = 1 warning: only found 1 / 8 columns around data row: 4. Filling remaining columns with `missing`
└ @ CSV C:\Users\Alan\.julia\packages\CSV\0Elut\src\file.jl:577
┌ Warning: thread = 1 warning: only found 1 / 8 columns around data row: 5. Filling remaining columns with `missing`
└ @ CSV C:\Users\Alan\.julia\packages\CSV\0Elut\src\file.jl:577
┌ Warning: thread = 1 warning: only found 1 / 8 columns around data row: 6. Filling remaining c

┌ Warning: thread = 1 warning: only found 1 / 8 columns around data row: 57. Filling remaining columns with `missing`
└ @ CSV C:\Users\Alan\.julia\packages\CSV\0Elut\src\file.jl:577
┌ Warning: thread = 1 warning: only found 1 / 8 columns around data row: 58. Filling remaining columns with `missing`
└ @ CSV C:\Users\Alan\.julia\packages\CSV\0Elut\src\file.jl:577
┌ Warning: thread = 1 warning: only found 1 / 8 columns around data row: 59. Filling remaining columns with `missing`
└ @ CSV C:\Users\Alan\.julia\packages\CSV\0Elut\src\file.jl:577
┌ Warning: thread = 1 warning: only found 1 / 8 columns around data row: 60. Filling remaining columns with `missing`
└ @ CSV C:\Users\Alan\.julia\packages\CSV\0Elut\src\file.jl:577
┌ Warning: thread = 1 warning: only found 1 / 8 columns around data row: 61. Filling remaining columns with `missing`
└ @ CSV C:\Users\Alan\.julia\packages\CSV\0Elut\src\file.jl:577
┌ Warning: thread = 1 warning: only found 1 / 8 columns around data row: 62. Filling remai

,dn
,String
1,"2.0,3.0,0.06293126448760776,0.03675776420360205,0.16002073300640293,""Dict{String, Real}(""""s_u"""" => 0.5, """"h_x_bar"""" => 0.1580972430397225, """"h_x"""" => 0.15809724303972247, """"p_u"""" => 0.5, """"s_y"""" => 0.5, """"a"""" => 2, """"h_y_bar"""" => 0.09224766111620283, """"M"""" => 1, """"phi"""" => 0.1, """"s_ay"""" => 0.5, """"b_x"""" => 7.0, """"h_y"""" => 0.09224766111620283, """"b_y"""" => 12.0, """"s_x"""" => 0.5, """"p_a"""" => 0.25, """"s_ax"""" => 0.5)"",css found"
2,"2.1578947368421053,3.0,0.08842857314511045,0.05077879533407955,0.15699275835717624,""Dict{String, Real}(""""s_u"""" => 0.5, """"h_x_bar"""" => 0.1580972430397225, """"h_x"""" => 0.15809724303972247, """"p_u"""" => 0.5, """"s_y"""" => 0.5, """"a"""" => 2, """"h_y_bar"""" => 0.09224766111620283, """"M"""" => 1, """"phi"""" => 0.1, """"s_ay"""" => 0.5, """"b_x"""" => 7.0, """"h_y"""" => 0.09224766111620283, """"b_y"""" => 12.0, """"s_x"""" => 0.5, """"p_a"""" => 0.25, """"s_ax"""" => 0.5)"",css found"
3,"2.3157894736842106,3.0,0.112043898641008,0.06276542995409165,0.15402114285386592,""Dict{String, Real}(""""s_u"""" => 0.5, """"h_x_bar"""" => 0.1580972430397225, """"h_x"""" => 0.15809724303972247, """"p_u"""" => 0.5, """"s_y"""" => 0.5, """"a"""" => 2, """"h_y_bar"""" => 0.09224766111620283, """"M"""" => 1, """"phi"""" => 0.1, """"s_ay"""" => 0.5, """"b_x"""" => 7.0, """"h_y"""" => 0.09224766111620283, """"b_y"""" => 12.0, """"s_x"""" => 0.5, """"p_a"""" => 0.25, """"s_ax"""" => 0.5)"",css found"
4,"2.473684210526316,3.0,0.13406181334290948,0.07313257964901168,0.15111009826743144,""Dict{String, Real}(""""s_u"""" => 0.5, """"h_x_bar"""" => 0.1580972430397225, """"h_x"""" => 0.15809724303972247, """"p_u"""" => 0.5, """"s_y"""" => 0.5, """"a"""" => 2, """"h_y_bar"""" => 0.09224766111620283, """"M"""" => 1, """"phi"""" => 0.1, """"s_ay"""" => 0.5, """"b_x"""" => 7.0, """"h_y"""" => 0.09224766111620283, """"b_y"""" => 12.0, """"s_x"""" => 0.5, """"p_a"""" => 0.25, """"s_ax"""" => 0.5)"",css found"
5,"2.6315789473684212,3.0,0.15468514986312565,0.08215618954666441,0.14826445975701108,""Dict{String, Real}(""""s_u"""" => 0.5, """"h_x_bar"""" => 0.1580972430397225, """"h_x"""" => 0.15809724303972247, """"p_u"""" => 0.5, """"s_y"""" => 0.5, """"a"""" => 2, """"h_y_bar"""" => 0.09224766111620283, """"M"""" => 1, """"phi"""" => 0.1, """"s_ay"""" => 0.5, """"b_x"""" => 7.0, """"h_y"""" => 0.09224766111620283, """"b_y"""" => 12.0, """"s_x"""" => 0.5, """"p_a"""" => 0.25, """"s_ax"""" => 0.5)"",css found"
6,"2.789473684210526,3.0,0.17408509500646677,0.09006444700205316,0.14548654950835244,""Dict{String, Real}(""""s_u"""" => 0.5, """"h_x_bar"""" => 0.1580972430397225, """"h_x"""" => 0.15809724303972247, """"p_u"""" => 0.5, """"s_y"""" => 0.5, """"a"""" => 2, """"h_y_bar"""" => 0.09224766111620283, """"M"""" => 1, """"phi"""" => 0.1, """"s_ay"""" => 0.5, """"b_x"""" => 7.0, """"h_y"""" => 0.09224766111620283, """"b_y"""" => 12.0, """"s_x"""" => 0.5, """"p_a"""" => 0.25, """"s_ax"""" => 0.5)"",css found"
7,"2.9473684210526314,3.0,0.19248780327619408,0.09708045352803482,0.14277156500888588,""Dict{String, Real}(""""s_u"""" => 0.5, """"h_x_bar"""" => 0.1580972430397225, """"h_x"""" => 0.15809724303972247, """"p_u"""" => 0.5, """"s_y"""" => 0.5, """"a"""" => 2, """"h_y_bar"""" => 0.09224766111620283, """"M"""" => 1, """"phi"""" => 0.1, """"s_ay"""" => 0.5, """"b_x"""" => 7.0, """"h_y"""" => 0.09224766111620283, """"b_y"""" => 12.0, """"s_x"""" => 0.5, """"p_a"""" => 0.25, """"s_ax"""" => 0.5)"",css found"
8,"3.1052631578947367,3.0,0.20991491916957594,0.10322111388557197,0.1401295909139746,""Dict{String, Real}(""""s_u"""" => 0.5, """"h_x_bar"""" => 0.1580972430397225, """"h_x"""" => 0.15809724303972247, """"p_u"""" => 0.5, """"s_y"""" => 0.5, """"a"""" => 2, """"h_y_bar"""" => 0.09224766111620283, """"M"""" => 1, """"phi"""" => 0.1, """"s_ay"""" => 0.5, """"b_x"""" => 7.0, """"h_y"""" => 0.09224766111620283, """"b_y"""" => 12.0, """"s_x"""" => 0.5, """"p_a

In [116]:
a=data[!,1][1][1]

'2': ASCII/Unicode U+0032 (category Nd: Number, decimal digit)

In [117]:
a

'2': ASCII/Unicode U+0032 (category Nd: Number, decimal digit)